In [147]:
import pandas as pd
import matplotlib.pylab as plt
import numpy as np
from scipy import stats
from sklearn.metrics import classification_report, accuracy_score

In [148]:
df = pd.read_csv('creditcardfraud_normalised.csv',nrows=10000)

In [149]:
# profile = ProfileReport(df, title="Credit Card Fraud")

In [150]:
# profile.to_file("CreditCardFraudEDA.html")

In [151]:
df.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,class
0,0.935192,0.766490,0.881365,0.313023,0.763439,0.267669,0.266815,0.786444,0.475312,0.510600,...,0.561184,0.522992,0.663793,0.391253,0.585122,0.394557,0.418976,0.312697,0.005824,0
1,0.978542,0.770067,0.840298,0.271796,0.766120,0.262192,0.264875,0.786298,0.453981,0.505267,...,0.557840,0.480237,0.666938,0.336440,0.587290,0.446013,0.416345,0.313423,0.000105,0
2,0.935217,0.753118,0.868141,0.268766,0.762329,0.281122,0.270177,0.788042,0.410603,0.513018,...,0.565477,0.546030,0.678939,0.289354,0.559515,0.402727,0.415489,0.311911,0.014739,0
3,0.941878,0.765304,0.868484,0.213661,0.765647,0.275559,0.266803,0.789434,0.414999,0.507585,...,0.559734,0.510277,0.662607,0.223826,0.614245,0.389197,0.417669,0.314371,0.004807,0
4,0.938617,0.776520,0.864251,0.269796,0.762975,0.263984,0.268968,0.782484,0.490950,0.524303,...,0.561327,0.547271,0.663392,0.401270,0.566343,0.507497,0.420561,0.317490,0.002724,0


In [152]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 30 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   V1      10000 non-null  float64
 1   V2      10000 non-null  float64
 2   V3      10000 non-null  float64
 3   V4      10000 non-null  float64
 4   V5      10000 non-null  float64
 5   V6      10000 non-null  float64
 6   V7      10000 non-null  float64
 7   V8      10000 non-null  float64
 8   V9      10000 non-null  float64
 9   V10     10000 non-null  float64
 10  V11     10000 non-null  float64
 11  V12     10000 non-null  float64
 12  V13     10000 non-null  float64
 13  V14     10000 non-null  float64
 14  V15     10000 non-null  float64
 15  V16     10000 non-null  float64
 16  V17     10000 non-null  float64
 17  V18     10000 non-null  float64
 18  V19     10000 non-null  float64
 19  V20     10000 non-null  float64
 20  V21     10000 non-null  float64
 21  V22     10000 non-null  float64
 22 

In [153]:
df['class'].sum()

38

In [154]:
%pip install combo

In [155]:
y = df['class'].copy()
X = df.drop(['class'], axis=1).copy()

In [156]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=69)
xx , yy = np.meshgrid(np.linspace(0,1 , 200), np.linspace(0, 1, 200))


In [157]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape


((8000, 29), (2000, 29), (8000,), (2000,))

In [158]:
from pyod.models.abod import ABOD
from pyod.models.knn import KNN
from pyod.models.iforest import IForest
from pyod.models.feature_bagging import FeatureBagging


outlier_fraction = 0.0038

classifiers = {
     'Angle-based Outlier Detector (ABOD)'   : ABOD(contamination=outlier_fraction),
     'K Nearest Neighbors (KNN)' :  KNN(contamination=outlier_fraction),
     'Isolation Forest': IForest(contamination=outlier_fraction,random_state=69),
     'Ensembling ': FeatureBagging(base_estimator=KNN(), contamination=outlier_fraction)
}


In [159]:
outlier_preds = {}

In [160]:
for name, clf in classifiers.items():
    clf.fit(X_train)
    y_test_pred = clf.predict(X_test)
    y_test_score = clf.decision_function(X_test)
    n_inliers = len(y_test_pred)-y_test_pred.sum()
    n_outliers = y_test_pred.sum()
    outlier_preds[name] = y_test_pred
    print(name, " : \n")
    print("Predicted inliers: ", n_inliers)
    print("Predicted outliers: ", n_outliers)
    print("Actual inliers: ", len(y_test)-y_test.sum())
    print("Actual Outliers: ", y_test.sum())
    print("Accuracy score on test data: ", accuracy_score(y_test, y_test_pred))
    print("===========================================================================")
    print("\n")

d:\ml-assignments\.venv\Lib\site-packages\numpy\core\fromnumeric.py:3787: RuntimeWarning: Degrees of freedom <= 0 for slice
  return _methods._var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
d:\ml-assignments\.venv\Lib\site-packages\numpy\core\_methods.py:163: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
d:\ml-assignments\.venv\Lib\site-packages\numpy\core\_methods.py:198: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
d:\ml-assignments\.venv\Lib\site-packages\numpy\core\fromnumeric.py:3787: RuntimeWarning: Degrees of freedom <= 0 for slice
  return _methods._var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
d:\ml-assignments\.venv\Lib\site-packages\numpy\core\_methods.py:163: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
d:\ml-assignments\.venv\Lib\site-packages\numpy\core\_methods.py:198: RuntimeWarning: invalid 

Angle-based Outlier Detector (ABOD)  : 

Predicted inliers:  2000
Predicted outliers:  0
Actual inliers:  1989
Actual Outliers:  11
Accuracy score on test data:  0.9945


K Nearest Neighbors (KNN)  : 

Predicted inliers:  1986
Predicted outliers:  14
Actual inliers:  1989
Actual Outliers:  11
Accuracy score on test data:  0.9925




d:\ml-assignments\.venv\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but IsolationForest was fitted without feature names
  warnings.warn(
d:\ml-assignments\.venv\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but IsolationForest was fitted without feature names
  warnings.warn(


Isolation Forest  : 

Predicted inliers:  1993
Predicted outliers:  7
Actual inliers:  1989
Actual Outliers:  11
Accuracy score on test data:  0.995


Ensembling   : 

Predicted inliers:  1988
Predicted outliers:  12
Actual inliers:  1989
Actual Outliers:  11
Accuracy score on test data:  0.9925


